# 1. 라이브러리 및 데이터

In [ ]:
# matplotlib 한글 깨짐방지
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (6,787 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 155222 files and d

In [83]:
import pandas as pd 
import numpy as np
import re 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
plt.rc('font', family='NanumBarunGothic') #matplotlib 한글 깨짐 방지

import warnings
warnings.filterwarnings(action='ignore')

In [84]:
# 데이터 불러오기
train=pd.read_csv("/content/movies_train.csv")
test=pd.read_csv("/content/movies_test.csv")
submission=pd.read_csv("/content/submission.csv")

# 2. 탐색적 자료분석(EDA)

### 1) 일부 데이터 미리보기

+ title : 영화의 제목
+ distributor : 배급사
+ genre : 장르
+ release_time : 개봉일
+ time : 상영시간(분)
+ screening_rat : 상영등급
+ director : 감독이름
+ dir_prev_bfnum : 해당 감독이 이 영화를 만들기 전 제작에 참여한 영화에서의 평균 관객수(단 관객수가 알려지지 않은 영화 제외)
+ dir_prev_num : 해당 감독이 이 영화를 만들기 전 제작에 참여한 영화의 개수(단 관객수가 알려지지 않은 영화 제외)
+ num_staff : 스텝수
+ num_actor : 주연배우수
+ box_off_num : 관객수

In [ ]:
train.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083
3,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866
4,불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387


In [ ]:
test.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor
0,용서는 없다,시네마서비스,느와르,2010-01-07,125,청소년 관람불가,김형준,3.005290e+05,2,304,3
1,아빠가 여자를 좋아해,(주)쇼박스,멜로/로맨스,2010-01-14,113,12세 관람가,이광재,3.427002e+05,4,275,3
2,하모니,CJ 엔터테인먼트,드라마,2010-01-28,115,12세 관람가,강대규,4.206611e+06,3,419,7
3,의형제,(주)쇼박스,액션,2010-02-04,116,15세 관람가,장훈,6.913420e+05,2,408,2
4,평행 이론,CJ 엔터테인먼트,공포,2010-02-18,110,15세 관람가,권호영,3.173800e+04,1,380,1


In [ ]:
submission.head()

,title,box_off_num
0,용서는 없다,0
1,아빠가 여자를 좋아해,0
2,하모니,0
3,의형제,0
4,평행 이론,0


### 2) 데이터 요약

In [ ]:
print(train.shape)
print(test.shape)
print(submission.shape)

(600, 12)
(243, 11)
(243, 2)


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           600 non-null    object 
 1   distributor     600 non-null    object 
 2   genre           600 non-null    object 
 3   release_time    600 non-null    object 
 4   time            600 non-null    int64  
 5   screening_rat   600 non-null    object 
 6   director        600 non-null    object 
 7   dir_prev_bfnum  270 non-null    float64
 8   dir_prev_num    600 non-null    int64  
 9   num_staff       600 non-null    int64  
 10  num_actor       600 non-null    int64  
 11  box_off_num     600 non-null    int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 56.4+ KB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           243 non-null    object 
 1   distributor     243 non-null    object 
 2   genre           243 non-null    object 
 3   release_time    243 non-null    object 
 4   time            243 non-null    int64  
 5   screening_rat   243 non-null    object 
 6   director        243 non-null    object 
 7   dir_prev_bfnum  107 non-null    float64
 8   dir_prev_num    243 non-null    int64  
 9   num_staff       243 non-null    int64  
 10  num_actor       243 non-null    int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 21.0+ KB


### 3) 데이터의 요약통계량 확인

In [ ]:
pd.options.display.float_format = '{:.1f}'.format

In [ ]:
train.describe()

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
count,600.0,270.0,600.0,600.0,600.0,600.0
mean,100.9,1050442.9,0.9,151.1,3.7,708181.8
std,18.1,1791408.3,1.2,165.7,2.4,1828005.9
min,45.0,1.0,0.0,0.0,0.0,1.0
25%,89.0,20380.0,0.0,17.0,2.0,1297.2
50%,100.0,478423.6,0.0,82.5,3.0,12591.0
75%,114.0,1286568.6,2.0,264.0,4.0,479886.8
max,180.0,17615314.0,5.0,869.0,25.0,14262766.0


In [ ]:
test.describe()

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor
count,243.0,107.0,243.0,243.0,243.0
mean,109.8,891669.5,0.9,159.3,3.5
std,124.0,1217341.5,1.2,163.0,2.1
min,40.0,34.0,0.0,0.0,0.0
25%,91.0,62502.0,0.0,18.0,2.0
50%,104.0,493120.0,0.0,105.0,3.0
75%,114.5,1080849.6,1.0,282.0,4.0
max,2015.0,6173099.5,6.0,776.0,16.0


### 4) 결측치 확인 및 데이터 전처리

In [ ]:
train.isna().sum()

title               0
distributor         0
genre               0
release_time        0
time                0
screening_rat       0
director            0
dir_prev_bfnum    330
dir_prev_num        0
num_staff           0
num_actor           0
box_off_num         0
dtype: int64

In [ ]:
test.isna().sum()

title               0
distributor         0
genre               0
release_time        0
time                0
screening_rat       0
director            0
dir_prev_bfnum    136
dir_prev_num        0
num_staff           0
num_actor           0
dtype: int64

+ train과 test 모두 dir_prev_bfnum 변수에 결측값이 존재한다.

In [ ]:
# 결측값의 특성 확인
train[train['dir_prev_bfnum'].isna()].head(3)

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,nan,0,91,2,23398
6,길위에서,백두대간,다큐멘터리,2013-05-23,104,전체 관람가,이창재,nan,0,32,5,53526
8,"1789, 바스티유의 연인들",유니버설픽쳐스인터내셔널코리아,뮤지컬,2014-09-18,129,전체 관람가,정성복,nan,0,3,5,4778


In [ ]:
test[test['dir_prev_bfnum'].isna()].head(3)

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor
5,회오리 바람,모쿠슈라,드라마,2010-02-25,95,15세 관람가,장건재,nan,0,156,2
6,경계도시 2,시네마 달,다큐멘터리,2010-03-18,104,15세 관람가,홍형숙,nan,0,11,2
7,이웃집 남자,(주)루믹스미디어,드라마,2010-03-18,100,청소년 관람불가,장동홍,nan,0,117,5


In [ ]:
print(train[train['dir_prev_bfnum'].isna()]['dir_prev_num'].sum())
print(test[test['dir_prev_bfnum'].isna()]['dir_prev_num'].sum())

0
0


+ train과 test 모두 dir_prev_bfnum이 결측값으로 존재하는 데이터들은 전부 dir_prev_num이 0인 영화들이다. 
+ 즉, 감독의 이전작품이 존재 하지 않은 경우이다. 

In [85]:
train['dir_prev_bfnum'].fillna(0, inplace=True)
test['dir_prev_bfnum'].fillna(0, inplace=True)

+ 따라서 dir_prev_bfnum에 있는 결측값은 전부 0으로 채운다.

In [86]:
train['release_time']=pd.to_datetime(train['release_time'])
test['release_time']=pd.to_datetime(test['release_time'])

+ release_time은 시간에 관련된 데이터인데 int형으로 인식되고 있다. release_time을 datetime형식으로 변환한다.

+ distributor 전처리

In [87]:
train['distributor'] = train.distributor.str.replace("(주)", '')
test['distributor'] = test.distributor.str.replace("(주)", '')

In [88]:
train['distributor'] = [re.sub(r'[^0-9a-zA-Z가-힣]', '', x) for x in train.distributor]
test['distributor'] = [re.sub(r'[^0-9a-zA-Z가-힣]', '', x) for x in test.distributor]

In [89]:
def get_dis(x) :
    if 'CJ' in x or 'CGV' in x :
        return 'CJ'
    elif '쇼박스' in x :
        return '쇼박스'
    elif 'SK' in x :
        return 'SK'
    elif '리틀빅픽' in x :
        return '리틀빅픽처스'
    elif '스폰지' in x :
        return '스폰지'
    elif '싸이더스' in x :
        return '싸이더스'
    elif '에이원' in x :
        return '에이원'
    elif '마인스' in x :
        return '마인스'
    elif '마운틴픽' in x :
        return '마운틴픽처스'
    elif '디씨드' in x :
        return '디씨드'
    elif '드림팩트' in x :
        return '드림팩트'
    elif '메가박스' in x :
        return '메가박스'
    elif '마운틴' in x :
        return '마운틴'
    else :
        return x

In [90]:
train['distributor'] = train.distributor.apply(get_dis)
test['distributor'] = test.distributor.apply(get_dis)

In [91]:
# 배급사별 median 관객수를 기준으로 rank encoding 
tr_nm_rank = train.groupby('distributor').box_off_num.median().reset_index(name = 'num_rank').sort_values(by = 'num_rank')
tr_nm_rank['num_rank'] = [i + 1 for i in range(tr_nm_rank.shape[0])]
tr_nm_rank

,distributor,num_rank
110,인피니티엔터테인먼트,1
15,고구마공작소,2
52,사람과사람들,3
97,위드시네마,4
19,나우콘텐츠,5
...,...,...
113,전망좋은영화사,143
105,이십세기폭스코리아,144
56,쇼박스,145
84,영구아트무비,146


+ genre 전처리

In [92]:
train.groupby('genre').box_off_num.mean().sort_values()

genre
뮤지컬       6.627000e+03
다큐멘터리     6.717226e+04
서스펜스      8.261100e+04
애니메이션     1.819267e+05
멜로/로맨스    4.259680e+05
미스터리      5.275482e+05
공포        5.908325e+05
드라마       6.256898e+05
코미디       1.193914e+06
SF        1.788346e+06
액션        2.203974e+06
느와르       2.263695e+06
Name: box_off_num, dtype: float64

In [93]:
# 장르별 평균 관객수를 기준으로 rank encoding 
train['genre_rank'] = train.genre.map({'뮤지컬' : 1, '다큐멘터리' : 2, '서스펜스' : 3, '애니메이션' : 4, '멜로/로맨스' : 5,
                                      '미스터리' : 6, '공포' : 7, '드라마' : 8, '코미디' : 9, 'SF' : 10, '액션' : 11, '느와르' : 12})
test['genre_rank'] = test.genre.map({'뮤지컬' : 1, '다큐멘터리' : 2, '서스펜스' : 3, '애니메이션' : 4, '멜로/로맨스' : 5,
                                      '미스터리' : 6, '공포' : 7, '드라마' : 8, '코미디' : 9, 'SF' : 10, '액션' : 11, '느와르' : 12})

+ 최종 데이터 병합

In [94]:
train = pd.merge(train, tr_nm_rank, how = 'left')
test = pd.merge(test, tr_nm_rank, how = 'left')

In [ ]:
train.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,genre_rank,num_rank
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,0.00,0,91,2,23398,11,134
1,내부자들,쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501,12,145
2,은밀하게 위대하게,쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083,11,145
3,나는 공무원이다,NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866,9,140
4,불량남녀,쇼박스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387,9,145


In [ ]:
test.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,genre_rank,num_rank
0,용서는 없다,시네마서비스,느와르,2010-01-07,125,청소년 관람불가,김형준,3.005290e+05,2,304,3,12,132.0
1,아빠가 여자를 좋아해,쇼박스,멜로/로맨스,2010-01-14,113,12세 관람가,이광재,3.427002e+05,4,275,3,5,145.0
2,하모니,CJ,드라마,2010-01-28,115,12세 관람가,강대규,4.206611e+06,3,419,7,8,141.0
3,의형제,쇼박스,액션,2010-02-04,116,15세 관람가,장훈,6.913420e+05,2,408,2,11,145.0
4,평행 이론,CJ,공포,2010-02-18,110,15세 관람가,권호영,3.173800e+04,1,380,1,7,141.0


In [ ]:
test.isna().sum()

title              0
distributor        0
genre              0
release_time       0
time               0
screening_rat      0
director           0
dir_prev_bfnum     0
dir_prev_num       0
num_staff          0
num_actor          0
genre_rank         0
num_rank          31
dtype: int64

In [95]:
test.fillna(0, inplace = True)

# 3. 데이터 시각화 및 분석




### 1) 관객수(box_off_num) 예측 


1.   RandomForestRegressor
2.   GradientBoostingRegressor
3.   XGBRegressor
4.   LGBMRegressor



+ 모델링 데이터 구성

In [96]:
X = train[['num_rank', 'time', 'num_staff', 'num_actor', 'genre_rank', 'screening_rat']] # 배급사, 상영시간, 스텝수, 주연배우수, 장르, 상영등급 
target = test[['num_rank', 'time', 'num_staff', 'num_actor', 'genre_rank', 'screening_rat']]
y = np.log1p(train.box_off_num) # target 값인 관객수 로그변환 

In [97]:
# 상영등급 One-Hot encoding 
X = pd.get_dummies(columns = ['screening_rat'], data = X) 
target = pd.get_dummies(columns = ['screening_rat'], data = target) 

In [98]:
# 주연배우수 로그변환 
X['num_actor'] = np.log1p(X['num_actor']) 
target['num_actor'] = np.log1p(target['num_actor']) 

In [ ]:
X

,num_rank,time,num_staff,num_actor,genre_rank,screening_rat_12세 관람가,screening_rat_15세 관람가,screening_rat_전체 관람가,screening_rat_청소년 관람불가
0,134,96,91,1.098612,11,0,0,0,1
1,145,130,387,1.386294,12,0,0,0,1
2,145,123,343,1.609438,11,0,1,0,0
3,140,101,20,1.945910,9,0,0,1,0
4,145,108,251,1.098612,9,0,1,0,0
...,...,...,...,...,...,...,...,...,...
595,140,111,510,2.079442,8,0,0,0,1
596,145,127,286,1.945910,8,0,1,0,0
597,46,99,123,1.609438,7,0,0,0,1
598,141,102,431,1.609438,12,0,1,0,0


+ 10Fold 교차검증

In [99]:
# 10개의 폴드 세트로 분리하는 KFold 객체와 폴드 세트별 rmse를 담을 리스트 객체 생성.
from sklearn.model_selection import KFold
kf = KFold(n_splits =  10, shuffle = True, random_state = 11)
rmse_list = []

+ RandomForestRegressor

In [100]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 11)

In [101]:
rf_pred = np.zeros((test.shape[0])) # RF로 예측된 submission data 관객수  

# KFold객체의 split( ) 호출하면 폴드 별 학습용, 검증용 테스트의 로우 인덱스를 array로 반환  
for train_idx, test_idx in kf.split(X, y) :
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx] # 학습 데이터 크기: 540 
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]     # 검증 데이터 크기: 60 
    #학습 및 예측
    rf.fit(X_train, y_train)
    # 관객수니까 0보다 작으면 0으로 처리 
    # 관객수 로그변환 했으니까 예측값 계산시에는 지수변환 수행
    pred = np.expm1([0 if x < 0 else x for x in rf.predict(X_test)])      # X_test로 예측된 관객수 -> rmse 계산에 사용 
    sub_pred = np.expm1([0 if x < 0 else x for x in rf.predict(target)])  # target(test)로 예측된 관객수 -> submission data 관객수에 적용 
    rmse = np.sqrt(mean_squared_error(y_test, pred))
    rmse_list.append(rmse)
    
    rf_pred += (sub_pred / 10)  # 10-fold 교차검증이어서 10으로 나눠줌 

# rmse_list의 개별 rmse를 합하여 평균 rmse 계산 
print('RF 평균 rmse: {0:.2f}'.format(np.mean(rmse_list)))

RF 평균 rmse: 906446.18


+ GridSearchCV - RF

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'n_estimators': [10,50,100,200], 'max_depth':[1,2,3], 'min_samples_split':[2,3]}
grid_rf = GridSearchCV(rf, param_grid=parameters, cv=10, refit=True)
# Train 데이터로 param_grid의 하이퍼 파라미터들을 순차적으로 학습/평가 .
grid_rf.fit(X_train, y_train)
print('GridSearchCV 최적 파라미터:', grid_rf.best_params_)

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 10}


In [ ]:
# GridSearchCV의 refit으로 이미 학습이 된 estimator 반환
estimator = grid_rf.best_estimator_
# GridSearchCV의 best_estimator_는 이미 최적 하이퍼 파라미터로 학습이 됨
pred = np.expm1([0 if x < 0 else x for x in estimator.predict(X_test)]) 
rmse = np.sqrt(mean_squared_error(y_test, pred))
rmse_list.append(rmse)
# rmse_list의 개별 rmse를 합하여 평균 rmse 계산 
print('RF 평균 rmse: {0:.2f}'.format(np.mean(rmse_list)))

RF 평균 rmse: 874207.04


+ GradientBoostingRegressor

In [102]:
from sklearn.ensemble import GradientBoostingRegressor
gbm = GradientBoostingRegressor(random_state = 11)

In [103]:
gbm_pred = np.zeros((test.shape[0])) # GBM로 예측된 submission data 관객수  

for train_idx, test_idx in kf.split(X, y) :
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx] 
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]     
    gbm.fit(X_train, y_train)
    pred = np.expm1([0 if x < 0 else x for x in gbm.predict(X_test)])      
    sub_pred = np.expm1([0 if x < 0 else x for x in gbm.predict(target)])  
    rmse = np.sqrt(mean_squared_error(y_test, pred))
    rmse_list.append(rmse)
    
    gbm_pred += (sub_pred / 10)  

print('GBM 평균 rmse: {0:.2f}'.format(np.mean(rmse_list)))

GBM 평균 rmse: 995114.04


+ GridSearchCV - GBM 

In [ ]:
parameters = {'n_estimators': [100,500], 'max_depth':[1,2,3], 'learning_rate':[0.01,0.05,0.1]}
grid_gbm = GridSearchCV(gbm, param_grid=parameters, cv=10, refit=True)
# Train 데이터로 param_grid의 하이퍼 파라미터들을 순차적으로 학습/평가 .
grid_gbm.fit(X_train, y_train)
print('GridSearchCV 최적 파라미터:', grid_gbm.best_params_)

GridSearchCV 최적 파라미터: {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100}


In [ ]:
# GridSearchCV의 refit으로 이미 학습이 된 estimator 반환
estimator = grid_gbm.best_estimator_
# GridSearchCV의 best_estimator_는 이미 최적 하이퍼 파라미터로 학습이 됨
pred = np.expm1([0 if x < 0 else x for x in estimator.predict(X_test)]) 
rmse = np.sqrt(mean_squared_error(y_test, pred))
rmse_list.append(rmse)
# rmse_list의 개별 rmse를 합하여 평균 rmse 계산 
print('GBM 평균 rmse: {0:.2f}'.format(np.mean(rmse_list)))

GBM 평균 rmse: 959088.27


+ XGBRegressor

In [104]:
from xgboost import XGBRegressor
xgb = XGBRegressor(random_state = 11)

In [105]:
xgb_pred = np.zeros((test.shape[0])) # XGB로 예측된 submission data 관객수  

for train_idx, test_idx in kf.split(X, y) :
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx] 
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]     
    xgb.fit(X_train, y_train)
    pred = np.expm1([0 if x < 0 else x for x in xgb.predict(X_test)])      
    sub_pred = np.expm1([0 if x < 0 else x for x in xgb.predict(target)])  
    rmse = np.sqrt(mean_squared_error(y_test, pred))
    rmse_list.append(rmse)
    
    xgb_pred += (sub_pred / 10)  

print('XGB 평균 rmse: {0:.2f}'.format(np.mean(rmse_list)))

[10:00:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:00:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:00:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:00:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:00:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:00:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:00:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:00:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

+ GridSearchCV - XGB

In [ ]:
parameters = {'n_estimators': [10,50,100,500,1000], 'learning_rate':[0.001,0.005,0.01,0.05,0.1,0.2]}
grid_xgb = GridSearchCV(xgb, param_grid=parameters, cv=10, refit=True)
# Train 데이터로 param_grid의 하이퍼 파라미터들을 순차적으로 학습/평가 .
grid_xgb.fit(X_train, y_train)
print('GridSearchCV 최적 파라미터:', grid_xgb.best_params_)

[09:16:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:16:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:16:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:16:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:16:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:16:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:16:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:16:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [ ]:
# GridSearchCV의 refit으로 이미 학습이 된 estimator 반환
estimator = grid_xgb.best_estimator_
# GridSearchCV의 best_estimator_는 이미 최적 하이퍼 파라미터로 학습이 됨
pred = np.expm1([0 if x < 0 else x for x in estimator.predict(X_test)]) 
rmse = np.sqrt(mean_squared_error(y_test, pred))
rmse_list.append(rmse)  
# rmse_list의 개별 rmse를 합하여 평균 rmse 계산 
print('XGB 평균 rmse: {0:.2f}'.format(np.mean(rmse_list)))

XGB 평균 rmse: 967616.58


+ LGBMRegressor

In [106]:
from lightgbm import LGBMRegressor
lgbm = LGBMRegressor(random_state = 11)

In [107]:
lgbm_pred = np.zeros((test.shape[0])) # LGBM로 예측된 submission data 관객수  

for train_idx, test_idx in kf.split(X, y) :
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx] 
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]     
    lgbm.fit(X_train, y_train)
    pred = np.expm1([0 if x < 0 else x for x in lgbm.predict(X_test)])      
    sub_pred = np.expm1([0 if x < 0 else x for x in lgbm.predict(target)])  
    rmse = np.sqrt(mean_squared_error(y_test, pred))
    rmse_list.append(rmse)
    
    lgbm_pred += (sub_pred / 10)  

print('LGBM 평균 rmse: {0:.2f}'.format(np.mean(rmse_list)))

LGBM 평균 rmse: 1045087.85


+ GridSearchCV - LGBM

In [ ]:
parameters = {'n_estimators': [10,50,100,500,1000], 'learning_rate':[0.001,0.005,0.01,0.05,0.1,0.2]}
grid_lgbm = GridSearchCV(lgbm, param_grid=parameters, cv=10, refit=True)
# Train 데이터로 param_grid의 하이퍼 파라미터들을 순차적으로 학습/평가 .
grid_lgbm.fit(X_train, y_train)
print('GridSearchCV 최적 파라미터:', grid_lgbm.best_params_)

GridSearchCV 최적 파라미터: {'learning_rate': 0.05, 'n_estimators': 100}


In [ ]:
# GridSearchCV의 refit으로 이미 학습이 된 estimator 반환
estimator = grid_lgbm.best_estimator_
# GridSearchCV의 best_estimator_는 이미 최적 하이퍼 파라미터로 학습이 됨
pred = np.expm1([0 if x < 0 else x for x in estimator.predict(X_test)])
rmse = np.sqrt(mean_squared_error(y_test, pred))
rmse_list.append(rmse)
# rmse_list의 개별 rmse를 합하여 평균 rmse 계산 
print('LGBM 평균 rmse: {0:.2f}'.format(np.mean(rmse_list)))

LGBM 평균 rmse: 1019015.43


+ 최종 예측 결과 

In [108]:
# 4가지 모델의 각 예측값 평균으로 test data(=submission data)의 최종 예측 관객수 도출 
submission['box_off_num'] = (rf_pred + gbm_pred + xgb_pred + lgbm_pred) / 4
test['box_off_num'] = submission['box_off_num']

In [109]:
# 영화 제목별 관객수 예측 결과 오름차순 정리 
submission.sort_values(by = 'box_off_num') 


,title,box_off_num
130,댄서김의 은밀한 교수법,1.901327e+01
39,REC 알이씨,2.009790e+01
139,화려한 외출,2.920285e+01
173,옹녀뎐,3.236667e+01
167,바리새인,3.543388e+01
...,...,...
135,용의자,3.965337e+06
229,베테랑,4.436559e+06
142,박수건달,5.373479e+06
179,군도: 민란의 시대,5.578518e+06


In [110]:
submission.to_csv('submission2.csv', index = False, encoding='utf-8-sig')

### 2) 상영등급(screening_rat) 예측 

1.   DecisionTreeClassifier
2.   LogisticRegression



+ train data, test data 생성 

In [ ]:
X_train = train[['num_rank', 'time', 'num_staff', 'num_actor', 'genre_rank', 'dir_prev_bfnum', 'dir_prev_num', 'box_off_num']] # 배급사, 상영시간, 스텝수, 주연배우수, 장르, 이전평균관객수, 이전작품개수, 관객수
X_test = test[['num_rank', 'time', 'num_staff', 'num_actor', 'genre_rank', 'dir_prev_bfnum', 'dir_prev_num', 'box_off_num']]
# screening_rat 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(train['screening_rat'])
y_train = encoder.transform(train['screening_rat'])
y_test = encoder.transform(test['screening_rat'])

In [ ]:
encoder.classes_  # 순서대로 0,1,2,3 으로 encoding됨 

array(['12세 관람가', '15세 관람가', '전체 관람가', '청소년 관람불가'], dtype=object)

+ StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler

# StandardScaler객체 생성
scaler = StandardScaler()
# StandardScaler 로 데이터 셋 변환. fit( ) 과 transform( ) 호출.  
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

+ DecisionTreeClassifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
# DecisionTreeClassifier 객체 생성 
dt_clf = DecisionTreeClassifier(random_state=0)
# 학습 수행 
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=0)

In [ ]:
# 학습이 완료된 DecisionTreeClassifier 객체에서 테스트 데이터 세트로 예측 수행. 
pred = dt_clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도: 0.4403


In [ ]:
from sklearn.metrics import confusion_matrix
# 앞절의 예측 결과인 pred와 실제 결과인 y_test의 Confusion Matrix출력
print('오차행렬:\n', confusion_matrix(y_test , pred))

오차행렬:
 [[ 8 17  5 10]
 [15 34  3 21]
 [ 7  9 16  5]
 [ 6 32  6 49]]


+ LogisticRegression 

In [ ]:
from sklearn.linear_model import LogisticRegression 
lr = LogisticRegression()
lr.fit(X_train , y_train)
pred = lr.predict(X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도: 0.5185


In [ ]:
print('오차행렬:\n', confusion_matrix(y_test , pred))

오차행렬:
 [[ 0 21  6 13]
 [ 0 38  7 28]
 [ 0  5 27  5]
 [ 0 31  1 61]]


LogisticRegression이 DecisionTreeClassifier보다는 높은 예측 정확도를 보임

### 3) 차원 축소 (상영등급)

1.   랜덤포레스트
2.   PCA로 변환 후 랜덤포레스트



+ RandomForestClassifier

In [ ]:
# 랜덤포레스트에 적용 
from sklearn.ensemble import RandomForestClassifier
rcf = RandomForestClassifier(n_estimators=100, random_state=42)
rcf.fit(X_train , y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# 정확도 결과
pred = rcf.predict(X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도: 0.5226


+ PCA로 변환한 데이터 세트에 RandomForestClassifier

In [ ]:
X = pd.merge(pd.DataFrame(X_train), pd.DataFrame(X_test),how='outer')

In [ ]:
# PCA 적용 95% : n_components=0.95
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
pca.fit(X)
X_pca = pca.transform(X)

In [ ]:
X_train = X_pca[:600]
X_test = X_pca[600:]

In [ ]:
# PCA 이후 랜덤포레스트 훈련
rcf = RandomForestClassifier(n_estimators=100, random_state=42)
rcf.fit(X_train , y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# 정확도 결과 
pred = rcf.predict(X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도: 0.4033


기존 랜덤포레스트보다 예측 정확도 떨어짐 